## Resources:
- kristenwidman's blog: http://www.kristenwidman.com/blog/33/how-to-write-a-bittorrent-client-part-1/
- BitTorrentSpecification: https://wiki.theory.org/BitTorrentSpecification
- Pytorrent: https://github.com/gallexis/pytorrent
- Lita’s bittorrent: https://github.com/lita/bittorrent

## Major imports:
- `bcoding`: library used for converting data to and from bencode. Bencode is a type of encoding used by torrents.
- `hashlib`: used for sha1 hashing used in torrenting.
- `tqdm`: to show the progress of a loop
- `socket`: A way of connecting two nodes on a network (used to send data to and from, in peers)
- `struct`: Using `pack` and `unpack` methods to convert bytes to integer and vice versa. 
- `requests`: Sending http and udp requests to trackers for peer information.

In [1]:
from bcoding import bdecode, bencode
import hashlib
from tqdm import tqdm
import socket
import struct
import requests

## Read from torrent file:

In [2]:
# reading buffer from torrent file
buffer = open('file.torrent', 'rb').read()

# Decoding the bencode encoded binary data.
torrent = bdecode(buffer)

In [8]:
# The torrent is a dictionary with the following keys.
torrent.keys()

dict_keys(['announce', 'announce-list', 'comment', 'created by', 'creation date', 'info'])

In [4]:
## Getting a list of trackers to connect to.

# The torrent information contained in the torrrent file 
# has a list of ip addresses of the trackers available.
# This list is referred to as the announce-list.
# The first element of the list if present in the value of the 'announce' key.

announce_list = []
if 'announce-list' in torrent.keys():
    [announce_list.extend(tracker_list) for tracker_list in torrent['announce-list']]
if 'announce' in torrent.keys():
    announce_list.append(torrent['announce'])

print(f'Total trackers: {len(announce_list)}')
print(announce_list)

Total trackers: 3
['https://torrent.ubuntu.com/announce', 'https://ipv6.torrent.ubuntu.com/announce', 'https://torrent.ubuntu.com/announce']


In [9]:
torrent['info'].keys()

dict_keys(['length', 'name', 'piece length', 'pieces'])

## Connecting to a tracker.
To connect to the tracker, we need to send the following info as an http get request:
- `info_hash`: the sha1 hash of the info dict in the torrent.
- `peer_id`: 20-byte string used as a unique ID for the client. 
- `left`: The number of bytes this client still has to download in base ten ASCII.
- `port`: The port number that the client is listening on.

In [29]:
info_hash = hashlib.sha1(bencode(torrent['info'])).digest()
peer_id = hashlib.sha1('nishantkr'.encode()).digest()

# Calculating the total number of bytes to download.
if 'length' in torrent['info'].keys():
    # Single file mode.
    total_length = torrent['info']['length']
else:
    # Multi file mode.
    total_length = sum([ x['length'] for x in torrent['info']['files'] ])

print(f'Total length of files to be downloaded in MB: {total_length/1024/1024}')

def request_http(tracker:str):
    '''
    Function that accepts tracker url, and returns the response.
    '''
    print(tracker)
    try:
        response = requests.get(tracker, params={
            'info_hash': info_hash,
            'peer_id': peer_id,
            'left': total_length,
            'uploaded': 0,
            'downloaded': 0,
            'port': 6881
        }, timeout=2,
        verify=False)
        response = bdecode(response.content)
        return response
    except Exception as e:
        print(e.__str__())
        return None

Total length of files to be downloaded in MB: 3649.55078125


In [30]:
# For now, we will only tackle plain http trackers.
response = None
for tracker in tqdm(announce_list):
    if tracker.startswith('http'):
        response = request_http(tracker)
    elif tracker.startswith('udp'):
        print('udp requests not parsing')
    else:
        print("{} Unknown protocol".format(tracker))

    if response is not None and 'complete' in response.keys():
        if response['complete'] > 0:
            print("Connected to tracker: {}".format(tracker))
            break
        else:
            print("Tracker does not contain a complete peer")

  0%|          | 0/3 [00:00<?, ?it/s]

https://torrent.ubuntu.com/announce


  0%|          | 0/3 [00:01<?, ?it/s]

Connected to tracker: https://torrent.ubuntu.com/announce


In [34]:
response

{'complete': 2180,
 'incomplete': 44,
 'interval': 1800,
 'peers': [{'ip': '2607:5300:60:623::1',
   'peer id': '-TR2940-ghf4ow7e211b',
   'port': 51413},
  {'ip': '2a01:480:2000:11::12',
   'peer id': '-TR2940-7o53jr2e33w5',
   'port': 51413},
  {'ip': '2607:5300:60:623::1',
   'peer id': '-TR2940-2wzyoqgc8nlj',
   'port': 51413},
  {'ip': '2602:fdb8:131:2001::4:1',
   'peer id': b'-lt0D80-]u\x06\xf1\xdb\x915\x8e\x1cq43',
   'port': 61036},
  {'ip': '2a09:be40:3104:10::1',
   'peer id': b'-lt0D70-\r\x8aLd>\xf9\n\xa7\x16oH\xa9',
   'port': 11530},
  {'ip': '2a02:8108:9bbf:e9f4:1a31:bfff:fe53:ef5f',
   'peer id': '-TR3000-mw7imq9shk3c',
   'port': 51413},
  {'ip': '2405:9800:b902:7f85:4009:7885:1327:e0d3',
   'peer id': '-TR400Z-98n9wvt10sdx',
   'port': 55538},
  {'ip': '2a0a:e340:1001:400::2:1',
   'peer id': b'-lt0D80-I\xedPTR\x0f\xe3\xc4\x81\x04k\x06',
   'port': 57803},
  {'ip': '2a0a:51c0:0:18d::22',
   'peer id': '-TR300Z-ywge309tinas',
   'port': 62304},
  {'ip': '2405:9800:b902

In [38]:
# Peers can have two kinds of response: (see here: https://wiki.theory.org/BitTorrentSpecification#Tracker_Response)

peer_list = []
if type(response['peers']) is not list:
    # binary model
    offset = 0
    address = response['peers']

    for _ in range(len(address)//6):
        ip = address[offset:offset+4]
        ip = socket.inet_ntoa(ip)
        offset+=4
        port = address[offset:offset+2]
        port = struct.unpack('!H', port)[0]
        offset+=2
        peer_list.append((ip, port))

else:
    # dictionary model.
    for peer in response['peers']:
        peer_list.append((peer['ip'], peer['port']))

peer_list

[('2607:5300:60:623::1', 51413),
 ('2a01:480:2000:11::12', 51413),
 ('2607:5300:60:623::1', 51413),
 ('2602:fdb8:131:2001::4:1', 61036),
 ('2a09:be40:3104:10::1', 11530),
 ('2a02:8108:9bbf:e9f4:1a31:bfff:fe53:ef5f', 51413),
 ('2405:9800:b902:7f85:4009:7885:1327:e0d3', 55538),
 ('2a0a:e340:1001:400::2:1', 57803),
 ('2a0a:51c0:0:18d::22', 62304),
 ('2405:9800:b902:7f85:f5ca:ff2d:d388:12d', 52873),
 ('2806:2f0:7081:fcc1:88b9:1f55:7048:49ea', 51413),
 ('2a02:168:f405::70', 51414),
 ('2604:3d08:7f7d:700:203:dd15:bf25:9bd8', 51413),
 ('2001:470:1f0b:3da:3d71:4b37:399d:c494', 51413),
 ('2409:8a14:c222:950:211:32ff:fec9:c076', 16881),
 ('2001:41d0:602:3361::', 51413),
 ('2600:1700:6681:2160:5c55:6a45:1c50:b5ed', 51413),
 ('2601:601:200:813:a8a1:59ff:fea0:13fe', 6881),
 ('2001:470:7a83:6f74:0:7069:7261:7465', 6979),
 ('2a01:c22:b021:f000:7cc2:b978:7903:92bc', 51413),
 ('2001:b07:646a:78a:3a38:bee1:4d00:82de', 3150),
 ('2001:470:1f11:5a5::dead:beef', 48001),
 ('185.125.190.59', 6948),
 ('2001:47

In [36]:
type(response['peers'])

list

In [44]:
for peer in tqdm(peer_list):
    try:
        s = socket.create_connection(peer, timeout = 2)
        print(f'Connection established with {peer}')
        
        break
    except Exception as e:
        print(e.__str__())
        pass

 16%|█▌        | 8/50 [00:00<00:00, 76.89it/s]

[WinError 10051] A socket operation was attempted to an unreachable network
[WinError 10051] A socket operation was attempted to an unreachable network
[WinError 10051] A socket operation was attempted to an unreachable network
[WinError 10051] A socket operation was attempted to an unreachable network
[WinError 10051] A socket operation was attempted to an unreachable network
[WinError 10051] A socket operation was attempted to an unreachable network
[WinError 10051] A socket operation was attempted to an unreachable network
[WinError 10051] A socket operation was attempted to an unreachable network
[WinError 10051] A socket operation was attempted to an unreachable network
[WinError 10051] A socket operation was attempted to an unreachable network
[WinError 10051] A socket operation was attempted to an unreachable network
[WinError 10051] A socket operation was attempted to an unreachable network
[WinError 10051] A socket operation was attempted to an unreachable network
[WinError 10

 38%|███▊      | 19/50 [00:00<00:00, 90.43it/s]

[WinError 10051] A socket operation was attempted to an unreachable network
[WinError 10051] A socket operation was attempted to an unreachable network
[WinError 10051] A socket operation was attempted to an unreachable network
[WinError 10051] A socket operation was attempted to an unreachable network
[WinError 10051] A socket operation was attempted to an unreachable network
[WinError 10051] A socket operation was attempted to an unreachable network
[WinError 10051] A socket operation was attempted to an unreachable network


 44%|████▍     | 22/50 [00:00<00:00, 42.72it/s]

Connection established with ('185.125.190.59', 6948)


In [45]:
#<pstrlen><pstr><reserved><info_hash><peer_id>
PSTR = 'BitTorrent protocol'
reserved = '\x00\x00\x00\x00\x00\x00\x00\x00'

handshake_message = '\x13'.encode()+PSTR.encode()+reserved.encode()
info_hash_index = len(handshake_message)
# handshake_message += peer_id+peer_id
handshake_message += info_hash+peer_id

In [46]:
s.send(handshake_message)

68

In [59]:
full_data = b''
while True:
    try:
        data = s.recv(1024)
    except Exception as e:
        print(e.__str__())
        break
    if not data:
        print('no data')
        break
    full_data += data
full_data

[WinError 10053] An established connection was aborted by the software in your host machine


b''

In [48]:
# Checking if info_hash received matches the sent info_hash. 
full_data[info_hash_index:info_hash_index+20] == info_hash

True

In [49]:
next_msg = full_data[len(handshake_message):]

In [50]:
next_msg

b'\x00\x00\x00\x01\x01\x00\x00\x00\x05\x04\x00\x00\x13\x03'

In [60]:
# Function to decode message:
def decode_message(msg):
    msg_length = struct.unpack('!L', next_msg[:4])[0]
    msg_id = next_msg[4]
    print(f'Message Length: {msg_length}, Message ID: {msg_id}')

decode_message(next_msg)

Message Length: 1, Message ID: 1


In [58]:
# Interested
s.send(b'\x00\x00\x00\x01\x02')

5

In [119]:
# Keep-alive
s.send(b'\x00'*4)

4

In [131]:
s.send(struct.pack('!iB', 1, 2))

5

In [770]:
struct.pack('!iB', 1, 2)

b'\x00\x00\x00\x01\x02'